In [14]:
import tensorflow as tf
from tensorflow.keras.callbacks import EarlyStopping
from transformers import AutoTokenizer, TFAutoModelForSequenceClassification

# Pretrained model
CHECKPOINT = 'bert-base-uncased'
tokenizer = AutoTokenizer.from_pretrained(CHECKPOINT)
pretrained_model = TFAutoModelForSequenceClassification.from_pretrained(CHECKPOINT)

Downloading:   0%|          | 0.00/536M [00:00<?, ?B/s]

All model checkpoint layers were used when initializing TFBertForSequenceClassification.

Some layers of TFBertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [15]:
# BERT fine-tuning with batched Dataset
# texts と labels から BERT 入力用の batch 化された Dataset を生成するミニマルな処理

# tokenize する
def tokenize(texts):
    '''BERT用tokenizerでtokenizeする
    
    Args:
        texts (list<str>): モデルに入力されるテキストのリスト
        
    Returns:
        dict: BERT用トークナイザの戻り値(後続処理のためにpaddingしたもの)
        
    '''
    return tokenizer(texts,
                     max_length=6,
                     truncation=True,
                     padding='max_length',
                     add_special_tokens=True,
                     return_tensors='tf')

def get_dataset(bert_tokenized_texts, labels):
    '''BERT用トークナイザでトークナイズされたテキストとラベル列からシンプルなDatasetを作る.
    dataset の列の並び順は任意だが, 次のステップの処理はこの並び順に依存する. 
    本コードでは input_ids, attention_mask, labels の順とする.
    
    Args:
        bert_tokenized_texts (dict): BERT用トークナイザの戻り値(後続処理のために要padding)
        labels (list): ラベル列
        
    Returns:
        tensorflow.data.Dataset: input_ids, attention_mask, labels 列を持つシンプルなデータセット
        
    '''
    return tf.data.Dataset.from_tensor_slices((bert_tokenized_texts['input_ids'],
                                               bert_tokenized_texts['attention_mask'],
                                               labels))

def get_bert_compatible_dataset(dataset):
    '''dataset を huggingface.transformers の BERT に入力可能な形式に変換する.
    
    Args:
        dataset (tensorflow.data.Dataset): input_ids, attention_mask, labels 列を
                                           持つシンプルなデータセット
    Returns:
        dataset (tensorflow.data.MapDataset): 各 instance が BERT の入力形式に変換されたデータセット
    
    '''

    def _get_bert_compatible_instance(input_ids, attention_mask, labels):
        '''Dataset.map() 用の関数. Dataset の各 instance を BERT の入力形式に変換する.
        引数の並び順は呼び出し元Datasetの列の並び順に依存している(前のステップで定められている).

        Args:
            input_ids (list): 
            attention_mask (list):
            labels (list):

        Returns:
            tuple: ({'input_ids': input_ids, 'attention_mask': attention_mask}, label) のタプル

        '''
        return {
            'input_ids': input_ids,
            'attention_mask': attention_mask
        }, labels
    return dataset.map(_get_bert_compatible_instance)

# batch 化する
def get_batched_dataset(dataset, batch_size):
    '''バッチ化されたデータセットを作成する
    
    Args:
        dataset (tensorflow.data.Dataset): 任意のデータセット
        batch_size (int): バッチサイズ
    Returns:
        tensorflow.data.BatchDataset: バッチ化されたデータセット
    
    '''
    return dataset.batch(batch_size=batch_size)


# 処理
def get_bert_batch_dataset(texts, labels, batch_size=32):
    '''テキスト列とラベル列からバッチ化されたBERT入力用のデータセットを作成する
    
    Args:
        texts (list): BERT の fine-tuning に用いるテキスト列
        labels (list): BERT の fine-tuning に用いるラベル列
        
    Returns:
        tensorflow.data.BatchDataset: バッチ化されたBERT入力用のデータセット
    
    '''
    bert_tokenized_texts = tokenize(texts)
    dataset = get_dataset(bert_tokenized_texts, labels)
    dataset = get_bert_compatible_dataset(dataset)
#     dataset = get_batched_dataset(dataset, batch_size=batch_size)
    return dataset

batch_size = 2

# **
# Create Dataset (Does the statement contains "cat" ?)
# *
texts_train = ['I like cat',
               'I do not like cat',
               'I like dog',
               'I do not like dog']
labels_train = [1,
                1,
                0,
                0]
texts_valid = ['I love cat',
               'I am cat',
               'I love dog',
               'I am dog']
labels_valid = [1,
                1,
                0,
                0]
texts_test = ['cat walked away from me',
              'I miss my dog']

ds_train = get_bert_batch_dataset(texts_train, labels_train, batch_size=batch_size)
ds_valid = get_bert_batch_dataset(texts_valid, labels_valid, batch_size=batch_size)

In [18]:
# **
# Construct model
# *
model = tf.keras.models.Sequential()
model.add(pretrained_model)
model.add(tf.keras.layers.Dense(2))

# BERT は最終層のみ Fine tuning
model.layers[0].layers[0].trainable = False
model.layers[0].layers[1].trainable = False
model.layers[0].layers[2].trainable = True

model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
              loss='accuracy',
              metrics='accuracy')

# **
# Train
# *
# Callbacks
# early_stopping = EarlyStopping(monitor='val_loss', mode='min', patience=4, verbose=0)

# # # Training
# history = model.fit(ds_train,
#                     epochs=2,
#                     batch_size=batch_size,
# #                     validation_data=ds_valid,
#                     callbacks=[early_stopping])

# # Learning curve
# pd.DataFrame({'train': history.history['loss'],
#               'valid': history.history['val_loss']}).plot()

# **
# Predict
# *
# for X, _ in ds_train:
#     print(model.predict_on_batch(X))